In [1]:
from qiskit import *
from qiskit.circuit.random import random_circuit
from qiskit.circuit.library import QFT
import networkx as nx
import numpy as np
from scipy import sparse

from operations_graph import OperationsGraph
from route_forcing import RouteForcing

In [3]:
def decompose_to_cx(qc):
    count = 0
    while qc.num_nonlocal_gates() != len(qc.get_instructions('cx')) and count < 20000000:
        qc = qc.decompose()
        count += 1
        if count == 20000000:
            print('Decomposition stopped by count!', flush=True)
            count = 0
    
    return qc

In [5]:
def get_depth(qubits, mapper):
    circ = QuantumCircuit(qubits, qubits)

    for op in mapper.code:
        grid = int(np.sqrt(qubits))
        if op == 'NEXT':
            continue
        if len(op) == 2:
            circ.h(grid*op[1][0] + op[1][1])
        elif op[1] == op[2]:
            circ.h(grid*op[1][0] + op[1][1])
        else:
            circ.swap(grid*op[1][0] + op[1][1], grid*op[2][0] + op[2][1])

    return circ.depth()

In [10]:
qubits = 1024

G = nx.grid_2d_graph(int(np.sqrt(qubits)), int(np.sqrt(qubits)))
A = nx.adjacency_matrix(G).toarray()

adj_list = []
for i in range(len(A)):
    for j in range(len(A[0])):
        if A[i][j] == 1:
            adj_list.append([i, j])

pos = {n:n for n in G.nodes()}
edge_fidelity = {e:1.0 for e in G.edges()} # edge fidelity of 1 in all edges
edge_error_rate = {e:1-edge_fidelity[e] for e in G.edges()}

nx.set_node_attributes(G, pos, 'pos')

In [11]:
# Hyperparameters of the Route-Forcing algorithm
p = 1
k = 0

In [12]:
# Random Circuit -- 0.3s

depth = 40
circ = random_circuit(qubits, depth, max_operands=2)

op_graph = OperationsGraph(qiskit_circuit=circ)

mapper = RouteForcing(op_graph=op_graph, topology=G)
initial_placement = mapper.place('trivial')

mapper.map(k=k, initial_placement=initial_placement, swap_penalization=-p, conv_steps=10000)

print('SWAP gates:', mapper.added_swaps)
print('Resulting depth:', get_depth(qubits, mapper)) # Obtained from Qiskit

SWAP gates: 324056
Resulting depth: 1322


In [11]:
# QFT -- 4.3s

circ = QFT(qubits)
circ = decompose_to_cx(circ)

op_graph = OperationsGraph(qiskit_circuit=circ)

mapper = RouteForcing(op_graph=op_graph, topology=G)
initial_placement = mapper.place('trivial')

mapper.map(k=k, initial_placement=initial_placement, swap_penalization=-p, conv_steps=10000)

print('SWAP gates:', mapper.added_swaps)
print('Resulting depth:', get_depth(qubits, mapper)) # Obtained from Qiskit

SWAP gates: 19328
Resulting depth: 2141
